# Join Analysis: New Transactions to Awards

This notebook:
1. Loads the most recent transaction and award datasets
2. Identifies new awards vs modifications in transactions (using eda-3 methodology)
3. Filters to only new transactions
4. Joins new transactions to awards by award_id
5. Analyzes match rate: how many transactions find their award vs not found

## Setup

In [1]:
1+1

2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import glob

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

Libraries imported successfully!


## Load Most Recent Datasets

In [3]:
def find_most_recent_file(data_dir, pattern):
    """Find the most recent file matching pattern in data_dir"""
    files = glob.glob(str(Path(data_dir) / pattern))
    if not files:
        return None
    # Sort by modification time, most recent first
    files.sort(key=lambda x: Path(x).stat().st_mtime, reverse=True)
    return files[0]

# Look for most recent normalized datasets
data_dir = Path('../data/awards')

transactions_file = find_most_recent_file(data_dir, 'transactions_normalized_*.json')
awards_file = find_most_recent_file(data_dir, 'awards_normalized_*.json')

print("=== Available Datasets ===")
if transactions_file:
    print(f"Most recent transactions: {Path(transactions_file).name}")
if awards_file:
    print(f"Most recent awards: {Path(awards_file).name}")

if not transactions_file or not awards_file:
    raise FileNotFoundError("Need both transaction and award datasets!")

=== Available Datasets ===
Most recent transactions: transactions_normalized_2026-01-08_22-52-58.json
Most recent awards: awards_normalized_2026-01-09_00-19-53.json


In [4]:
# Load transactions
print("=== Loading Transactions ===")
with open(transactions_file, 'r') as f:
    transactions_data = json.load(f)

transactions_df = pd.DataFrame(transactions_data)

print(f"Loaded {len(transactions_df):,} transactions")
print(f"Shape: {transactions_df.shape}")
print(f"Columns: {transactions_df.shape[1]}")

=== Loading Transactions ===
Loaded 7,952 transactions
Shape: (7952, 22)
Columns: 22


In [5]:
# Load awards
print("=== Loading Awards ===")
with open(awards_file, 'r') as f:
    awards_data = json.load(f)

awards_df = pd.DataFrame(awards_data)

print(f"Loaded {len(awards_df):,} awards")
print(f"Shape: {awards_df.shape}")
print(f"Columns: {awards_df.shape[1]}")

=== Loading Awards ===
Loaded 1,000 awards
Shape: (1000, 18)
Columns: 18


In [7]:
awards_df.award_date.nunique()

1

In [ ]:
awards_df.award_amount > 9

## Identify New Awards vs Modifications

Following the methodology from eda-3.ipynb:
- **New awards**: `modification_number == '0'` OR `action_type_description == 'NEW'`
- **Modifications**: Everything else

In [ ]:
# Check key fields
print("=== Transaction Fields for Filtering ===")
print(f"\nModification Number - Top 10:")
print(transactions_df['modification_number'].value_counts().head(10))

print(f"\nAction Type Description - All values:")
print(transactions_df['action_type_description'].value_counts())

In [ ]:
# Apply the filter from eda-3
new_transactions = transactions_df[
    (transactions_df['modification_number'] == '0') | 
    (transactions_df['action_type_description'] == 'NEW')
]

modifications = transactions_df[
    (transactions_df['modification_number'] != '0') & 
    (transactions_df['action_type_description'] != 'NEW')
]

print("=== Transaction Filtering Results ===")
print(f"Total transactions: {len(transactions_df):,}")
print(f"New transactions: {len(new_transactions):,} ({len(new_transactions)/len(transactions_df)*100:.1f}%)")
print(f"Modifications: {len(modifications):,} ({len(modifications)/len(transactions_df)*100:.1f}%)")

print(f"\nNew transactions total value: ${new_transactions['federal_action_obligation'].sum():,.2f}")
print(f"New transactions avg value: ${new_transactions['federal_action_obligation'].mean():,.2f}")

In [ ]:
transactions_df.federal_action_obligation.sort_values(ascending=True)

In [ ]:
transactions_df[transactions_df.federal_action_obligation > 100_000].federal_action_obligation.sort_values(ascending=True)
# new_transactions.federal_action_obligation > 0.sort_values(ascending=True)

## Preview: New Transactions

In [ ]:
# Show sample of new transactions
print("=== Sample New Transactions ===")
new_transactions[[
    'transaction_id', 'award_id', 'action_date', 'modification_number',
    'action_type_description', 'federal_action_obligation', 
    'recipient_name', 'award_description'
]].head(10)

## Join New Transactions to Awards

Now we'll join the new transactions to awards by `award_id` and analyze the match rate.

In [ ]:
# Check unique award_ids in each dataset
print("=== Award ID Coverage ===")
print(f"Unique award_ids in new transactions: {new_transactions['award_id'].nunique():,}")
print(f"Unique award_ids in awards dataset: {awards_df['award_id'].nunique():,}")

# Check for overlaps
transaction_award_ids = set(new_transactions['award_id'])
awards_award_ids = set(awards_df['award_id'])

overlap = transaction_award_ids.intersection(awards_award_ids)
only_in_transactions = transaction_award_ids - awards_award_ids
only_in_awards = awards_award_ids - transaction_award_ids

print(f"\nAward IDs in both datasets: {len(overlap):,}")
print(f"Award IDs only in transactions: {len(only_in_transactions):,}")
print(f"Award IDs only in awards: {len(only_in_awards):,}")

In [ ]:
# Perform the join
print("=== Performing Left Join ===")
print("Joining new_transactions to awards on award_id...")

joined_df = new_transactions.merge(
    awards_df,
    on='award_id',
    how='left',
    suffixes=('_transaction', '_award'),
    indicator=True
)

print(f"\nJoined dataframe shape: {joined_df.shape}")
print(f"Columns: {joined_df.shape[1]}")

In [ ]:
joined_df.head()

In [ ]:
joined_df.source_url_award

In [ ]:
pd.set_option("display.max_colwidth", None)   # show full text in cells
pd.set_option("display.width", 200)            # total display width
pd.set_option("display.max_columns", None)     # show all columns
pd.DataFrame(joined_df.award_description_award)

In [ ]:
joined_df[joined_df.award_description_transaction != joined_df.award_description_award]

In [ ]:
tmp = joined_df.federal_action_obligation - joined_df.award_amount > 0

In [ ]:
tmp2 = joined_df.federal_action_obligation - joined_df.award_amount

In [ ]:
# joined_df[tmp]
tmp2[tmp]

In [ ]:
transactions_df.award_type

In [ ]:
transactions_df.groupby('award_type').count()

## Analyze Match Rate

In [ ]:
# Analyze the _merge indicator
print("=== Join Results Analysis ===")
print("\nMerge indicator breakdown:")
print(joined_df['_merge'].value_counts())

# Calculate match rates
matched = joined_df[joined_df['_merge'] == 'both']
not_matched = joined_df[joined_df['_merge'] == 'left_only']

print(f"\n=== MATCH RATE SUMMARY ===")
print(f"Total new transactions: {len(new_transactions):,}")
print(f"Transactions that found their award: {len(matched):,} ({len(matched)/len(new_transactions)*100:.1f}%)")
print(f"Transactions that did NOT find their award: {len(not_matched):,} ({len(not_matched)/len(new_transactions)*100:.1f}%)")

In [ ]:
# Analyze by value
matched_value = matched['federal_action_obligation'].sum()
not_matched_value = not_matched['federal_action_obligation'].sum()
total_value = new_transactions['federal_action_obligation'].sum()

print("=== MATCH RATE BY VALUE ===")
print(f"Total value of new transactions: ${total_value:,.2f}")
print(f"Value with matched awards: ${matched_value:,.2f} ({matched_value/total_value*100:.1f}%)")
print(f"Value without matched awards: ${not_matched_value:,.2f} ({not_matched_value/total_value*100:.1f}%)")

## Visualize Match Rate

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# By count
counts = [len(matched), len(not_matched)]
labels = [f'Found Award\n({len(matched):,})', f'Award Not Found\n({len(not_matched):,})']
colors = ['#2ecc71', '#e74c3c']
axes[0].pie(counts, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
axes[0].set_title('Transaction to Award Match Rate (Count)', fontsize=14, fontweight='bold')

# By value
values = [matched_value, not_matched_value]
values_b = [v/1e9 for v in values]
labels_val = [f'Found Award\n(${values_b[0]:.1f}B)', f'Award Not Found\n(${values_b[1]:.1f}B)']
axes[1].pie(values, labels=labels_val, autopct='%1.1f%%', colors=colors, startangle=90)
axes[1].set_title('Transaction to Award Match Rate (Value)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Analyze Unmatched Transactions

Why didn't some transactions find their awards?

In [ ]:
print("=== UNMATCHED TRANSACTIONS ANALYSIS ===")
print(f"\nTotal unmatched: {len(not_matched):,}")

# Show some examples
print("\nSample unmatched transactions:")
print(not_matched[[
    'transaction_id', 'award_id', 'action_date', 'modification_number',
    'federal_action_obligation', 'recipient_name', 'award_type_transaction'
]].head(20))

In [ ]:
# Analyze unmatched by award type
print("=== Unmatched Transactions by Award Type ===")
unmatched_by_type = not_matched.groupby('award_type_transaction').agg({
    'transaction_id': 'count',
    'federal_action_obligation': 'sum'
}).sort_values('transaction_id', ascending=False)
unmatched_by_type.columns = ['Count', 'Total Value']
print(unmatched_by_type)

In [ ]:
# Check dates - maybe the awards are from different time periods?
print("=== Date Range Comparison ===")
print(f"\nTransactions date range:")
print(f"  Min: {transactions_df['action_date'].min()}")
print(f"  Max: {transactions_df['action_date'].max()}")

print(f"\nAwards date range (award_date):")
# Filter out empty strings
award_dates = awards_df[awards_df['award_date'] != '']['award_date']
if len(award_dates) > 0:
    print(f"  Min: {award_dates.min()}")
    print(f"  Max: {award_dates.max()}")
    print(f"  Non-empty award_dates: {len(award_dates)} / {len(awards_df)}")
else:
    print("  No non-empty award_dates found!")

print(f"\nAwards date range (start_date):")
print(f"  Min: {awards_df['start_date'].min()}")
print(f"  Max: {awards_df['start_date'].max()}")

## Analyze Matched Transactions

For transactions that found their award, let's compare the data between transaction and award.

In [ ]:
print("=== MATCHED TRANSACTIONS ANALYSIS ===")
print(f"Total matched: {len(matched):,}")

# Show some examples
print("\nSample matched transactions with awards:")
print(matched[[
    'transaction_id', 'award_id', 'action_date', 
    'federal_action_obligation', 'award_amount',
    'recipient_name_transaction', 'recipient_name_award'
]].head(10))

In [ ]:
# Compare amounts: transaction vs award
# Note: transaction amount is the action amount, award amount is total award amount
print("=== Transaction vs Award Amounts ===")
print(f"\nTransaction amounts (federal_action_obligation):")
print(f"  Mean: ${matched['federal_action_obligation'].mean():,.2f}")
print(f"  Median: ${matched['federal_action_obligation'].median():,.2f}")
print(f"  Total: ${matched['federal_action_obligation'].sum():,.2f}")

print(f"\nAward amounts (award_amount):")
print(f"  Mean: ${matched['award_amount'].mean():,.2f}")
print(f"  Median: ${matched['award_amount'].median():,.2f}")
print(f"  Total: ${matched['award_amount'].sum():,.2f}")

print(f"\nNote: Award amounts are typically larger because they represent")
print(f"the total obligated amount for the entire award, while transaction")
print(f"amounts represent individual actions.")

In [ ]:
# Check if recipient names match
matched['recipient_match'] = matched['recipient_name_transaction'] == matched['recipient_name_award']

print("=== Recipient Name Consistency ===")
print(f"Transactions where recipient names match: {matched['recipient_match'].sum():,} ({matched['recipient_match'].sum()/len(matched)*100:.1f}%)")
print(f"Transactions where recipient names differ: {(~matched['recipient_match']).sum():,} ({(~matched['recipient_match']).sum()/len(matched)*100:.1f}%)")

# Show examples of mismatches
if (~matched['recipient_match']).sum() > 0:
    print("\nExamples of mismatched recipients:")
    mismatched = matched[~matched['recipient_match']][[
        'award_id', 'recipient_name_transaction', 'recipient_name_award'
    ]].head(10)
    print(mismatched.to_string())

## Summary & Key Findings

In [ ]:
print("="*80)
print("KEY FINDINGS: New Transactions to Awards Join Analysis")
print("="*80)

print(f"\n1. DATA VOLUMES")
print(f"   - Total transactions loaded: {len(transactions_df):,}")
print(f"   - New transactions (filtered): {len(new_transactions):,} ({len(new_transactions)/len(transactions_df)*100:.1f}%)")
print(f"   - Total awards loaded: {len(awards_df):,}")

print(f"\n2. MATCH RATE")
print(f"   - Transactions that found their award: {len(matched):,} ({len(matched)/len(new_transactions)*100:.1f}%)")
print(f"   - Transactions that did NOT find award: {len(not_matched):,} ({len(not_matched)/len(new_transactions)*100:.1f}%)")

print(f"\n3. MATCH RATE BY VALUE")
print(f"   - Total value (new transactions): ${total_value:,.2f}")
print(f"   - Matched value: ${matched_value:,.2f} ({matched_value/total_value*100:.1f}%)")
print(f"   - Unmatched value: ${not_matched_value:,.2f} ({not_matched_value/total_value*100:.1f}%)")

print(f"\n4. AWARD ID OVERLAP")
print(f"   - Unique award_ids in new transactions: {new_transactions['award_id'].nunique():,}")
print(f"   - Unique award_ids in awards: {awards_df['award_id'].nunique():,}")
print(f"   - Award IDs in both datasets: {len(overlap):,}")
print(f"   - Award IDs only in transactions: {len(only_in_transactions):,}")

print(f"\n5. DATA CONSISTENCY (for matched records)")
if len(matched) > 0:
    print(f"   - Recipient names match: {matched['recipient_match'].sum():,} ({matched['recipient_match'].sum()/len(matched)*100:.1f}%)")
    print(f"   - Recipient names differ: {(~matched['recipient_match']).sum():,} ({(~matched['recipient_match']).sum()/len(matched)*100:.1f}%)")

print(f"\n6. IMPLICATIONS")
if len(not_matched) > 0:
    not_matched_pct = len(not_matched)/len(new_transactions)*100
    if not_matched_pct > 50:
        print(f"   ⚠️  HIGH MISMATCH RATE: {not_matched_pct:.1f}% of new transactions don't find awards")
        print(f"   - Likely cause: Different time periods or different filtering criteria")
        print(f"   - Awards endpoint uses award-level summaries (rolled up)")
        print(f"   - Transactions endpoint has transaction-level detail")
    else:
        print(f"   ✓ GOOD MATCH RATE: {100-not_matched_pct:.1f}% of new transactions find their awards")
else:
    print(f"   ✓ PERFECT MATCH: All new transactions found their awards!")

print("\n" + "="*80)

## Export Results

In [ ]:
# Export matched and unmatched for further investigation
output_dir = Path('../data/awards')

matched_output = output_dir / 'matched_transactions_to_awards.csv'
unmatched_output = output_dir / 'unmatched_transactions.csv'

# Export matched (with selected columns to avoid duplication)
matched_export = matched[[
    'transaction_id', 'award_id', 'action_date', 'modification_number',
    'federal_action_obligation', 'award_amount',
    'recipient_name_transaction', 'recipient_name_award',
    'award_type_transaction', 'awarding_agency_name_transaction'
]]
matched_export.to_csv(matched_output, index=False)
print(f"Exported {len(matched):,} matched records to: {matched_output}")

# Export unmatched
unmatched_export = not_matched[[
    'transaction_id', 'award_id', 'action_date', 'modification_number',
    'federal_action_obligation', 'recipient_name_transaction',
    'award_type_transaction', 'awarding_agency_name_transaction',
    'award_description'
]]
unmatched_export.to_csv(unmatched_output, index=False)
print(f"Exported {len(not_matched):,} unmatched records to: {unmatched_output}")